<a href="https://colab.research.google.com/github/URIB4u/ATER_Finder/blob/main/ATER%20positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ATER Job Opportunity Finder

Welcome! This notebook helps you find "Attaché Temporaire d'Enseignement et de Recherche" (ATER) job opportunities in French higher education.

Follow the steps below to use the tool. You don't need any programming knowledge! Just run each section by clicking the "Play" button (the triangle) to the left of each cell and fill in the information when prompted.

## 1. Set Up the Tool

This section prepares the notebook by importing the necessary software tools and setting up some initial information. **Click the "Play" button to the left to run this section.** You should see some output indicating that the tools have been loaded.

In [1]:
# ----------------------------------------------
# This cell imports the necessary libraries and sets up the configuration for the script.
# It defines the URL for downloading the official Excel file, a fallback CSV file in case the download fails,
# and the column names for filtering the data later.


import requests
import pandas as pd
from io import BytesIO
import sys

# Columns we are interested in
columns_of_interest = [
    'Etablissement de rattachement',  # Institution
    'Profil appel à candidatures',     # Job profile
    'Date cloture candidature',        # Application deadline
    'Localisation appel à candidatures',  # Location
    'Quotité du poste',                  # Workload
    'Etat du poste',                      # Status
    'Type du poste'                      # Type of position
]

# --- Configuration ---
# URL of the official Excel file
ater_url = "https://www.galaxie.enseignementsup-recherche.gouv.fr/ensup/ATERListesOffresPubliees/ATEROffres_publiees_annee.xls"
# Fallback local CSV filename (if URL download fails)
fallback_csv = "https://raw.githubusercontent.com/URIB4u/ATER_Finder/refs/heads/main/ATER.csv"

# Columns we are interested in
profile_col = 'Profil appel à candidatures'
date_col = 'Date cloture candidature'

print("Libraries imported and configuration set.")
print(f"Configured URL: {ater_url}")
print(f"Fallback CSV: {fallback_csv}")
print(f"Profile column: {profile_col}")
print(f"Date column: {date_col}")

Libraries imported and configuration set.
Configured URL: https://www.galaxie.enseignementsup-recherche.gouv.fr/ensup/ATERListesOffresPubliees/ATEROffres_publiees_annee.xls
Fallback CSV: https://raw.githubusercontent.com/URIB4u/ATER_Finder/refs/heads/main/ATER.csv
Profile column: Profil appel à candidatures
Date column: Date cloture candidature


## 2. Load the Job Data

This section contains the instructions for the notebook to go and get the latest job listings from the internet. **Click the "Play" button to the left to run this section.** If the main source is unavailable, it will try to use a backup file.

In [2]:
def load_and_filter_data():
    """
    This function loads the data from the official URL or a fallback CSV file.
    It filters the data to include only the columns of interest.
    """
    data = None
    try:
        # Attempt to download the Excel file
        response = requests.get(ater_url)
        response.raise_for_status()  # Raise an error for bad status codes
        data = pd.read_excel(BytesIO(response.content), header=2) # Header is in the 3rd row (index 2)
        print("Data successfully loaded from URL.")

        # Drop the first two rows of data (which were originally rows 0 and 1)
        data = data.drop(index=[0, 1]).reset_index(drop=True)

    except Exception as e:
        print(f"Failed to load data from URL: {e}")
        print(f"Attempting to load fallback CSV: {fallback_csv}")
        try:
            data = pd.read_csv(fallback_csv)
            print("Data successfully loaded from fallback CSV.")
        except Exception as e:
            print(f"Failed to load fallback CSV: {e}")
            sys.exit("Unable to load data. Exiting.")

    if data is not None:
        # Clean column names (optional, if needed)
        data.columns = data.columns.str.strip()

        # Filter the DataFrame to include only the columns of interest
        try:
            filtered_data = data[columns_of_interest]
        except KeyError as e:
            print(f"Error: {e}")
            print("Please verify that the column names in 'columns_of_interest' match the actual column names in the loaded data.")
            print(f"Actual columns: {data.columns.tolist()}")
            sys.exit("Exiting due to column mismatch.")

        return filtered_data
    else:
        sys.exit("Failed to load data from both URL and fallback CSV.")

## 3. Process the Data

This section takes the job data that was loaded and prepares it for searching. **Click the "Play" button to the left to run this section.** You might not see any output here, but the data is being processed behind the scenes.

In [3]:
# Load the data
df = load_and_filter_data()

Data successfully loaded from URL.


## 4. Enter Your Search Criteria

Please use the form below to specify what kind of job you are looking for and the latest closing date you are interested in.

* **Enter Keyword (French):** Type the keyword related to your field of interest in French (e.g., informatique, droit privé).
* **Enter Closing Date (YYYY-MM-DD):** Enter the closing date you are interested in, using the format YYYY-MM-DD (e.g., 2025-05-15). The search will find positions closing on or after this date.

**After filling out the form, click the "Play" button to the left of the code cell below.**

In [5]:
# --- User Input (Colab Form) ---
keyword = "informatique"  # Default value, can be changed in the form
target_date_str = "2025-04-30"  # Default value, can be changed in the form

profile_col = 'Profil appel à candidatures'
date_col = 'Date cloture candidature'

try:
    target_date = pd.to_datetime(target_date_str)
except ValueError:
    print("Invalid date format. Please use YYYY-MM-DD.")
    sys.exit()

## 5. Find Matching Jobs

This section will now search through the job data using the keyword and date you provided. **Click the "Play" button to the left to run this section.** The results will be displayed below.

In [6]:
# --- Filtering and Display ---
if df is not None and 'keyword' in locals() and 'target_date' in locals(): # Check if variables are defined
    print(f"\n--- Filtering Results ---")
    print(f"Filtering for keyword '{keyword}' in '{profile_col}'")
    print(f"Filtering for closing date on or after {target_date.strftime('%d/%m/%Y')} in '{date_col}'")

    # Apply keyword filter (case-insensitive)
    # 'na=False' ensures rows with NaN in the profile column are excluded
    keyword_filter = df[profile_col].str.contains(keyword, case=False, na=False)

    # Apply date filter
    # Ensure the date column is in datetime format before comparison
    df[date_col] = pd.to_datetime(df[date_col], format='%d/%m/%Y', errors='coerce') # Specify the date format
    date_filter = (df[date_col] >= target_date) & (df[date_col].notna())

    # Combine filters
    filtered_df = df[keyword_filter & date_filter]

    # --- Display Results ---
    if not filtered_df.empty:
        print(f"\nFound {len(filtered_df)} matching positions:")
        for index, row in filtered_df.iterrows():
            print("\n-------------------- Position --------------------")
            display_cols = [col for col in columns_of_interest if col in filtered_df.columns]
            for col in display_cols:
                print(f"{col}: {row[col]}")
        print("\n-------------------- End of Results --------------------")
    else:
        print("\nNo positions found matching your criteria.")

elif df is None:
    print("\nCannot filter because data was not loaded.")
else:
    print("\nFiltering skipped because user input was not provided or was invalid.")


--- Filtering Results ---
Filtering for keyword 'informatique' in 'Profil appel à candidatures'
Filtering for closing date on or after 30/04/2025 in 'Date cloture candidature'

Found 1 matching positions:

-------------------- Position --------------------
Etablissement de rattachement: INSA DE RENNES
Profil appel à candidatures: INFORMATIQUE
Date cloture candidature: 2025-05-05 00:00:00
Localisation appel à candidatures: INSA RENNES
Quotité du poste: P
Etat du poste: V
Type du poste: ATER

-------------------- End of Results --------------------


## 6. Save Results to Excel (Optional)

If you would like to save the list of matching job opportunities to an Excel file on your Google Drive, click the "Play" button to the left of the code cell below and follow the instructions. You will be asked to enter a name for your file.

In [7]:
# --- Save Results to Excel ---
if 'filtered_df' in locals() and not filtered_df.empty:
    save_excel = input("\nDo you want to save these results to a single Excel sheet? (yes/no) or (y/n): ").lower()
    if save_excel in ['yes', 'y']:
        base_filename = input("Enter the filename for the Excel file (e.g., ater_positions): ")
        output_filename = f"{base_filename}.xlsx"
        try:
            filtered_df.to_excel(output_filename, sheet_name='All Results', index=False)
            print(f"\nResults saved to '{output_filename}' on the 'All Results' sheet.")
        except Exception as e:
            print(f"\nError saving to Excel: {e}")
else:
    print("\nNo results to save.")


Do you want to save these results to a single Excel sheet? (yes/no) or (y/n): y
Enter the filename for the Excel file (e.g., ater_positions): ater2025

Results saved to 'ater2025.xlsx' on the 'All Results' sheet.


In [ ]:
## End of Tool

You have reached the end of the ATER Job Opportunity Finder. If you encountered any issues or have suggestions for improvement, please feel free to provide feedback.